In [1]:
# %%

import pandas as pd
import numpy as np
from tabulate import tabulate

from CortesAnalysisPackage import plot as pt
from CortesAnalysisPackage import peakfitting as pfa
from CortesAnalysisPackage import componentfitting as ca
from CortesAnalysisPackage import classical as cla
# Import
import pandas as pd
import numpy as np
import INS_Analysis as insd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import linregress

df = pd.read_pickle("DetectorReadings.pkl")
bins = df['bins'].values
df = df.drop(columns=['bins'])

exp_df = pd.read_pickle('ExpirementData.pkl')

In [2]:
test_names = []
mses = []
r2s = []
predicted_concentrations = []

train_index = [22, 23]
test_index = np.arange(len(exp_df))
test_index = np.delete(test_index, train_index)
true_c_concentrations = exp_df['avg_carbon_portion'].values

In [3]:
elem_maps = exp_df['elem_maps'].to_list()
elem_maps = np.array(elem_maps)
mixed_ratios = elem_maps
mixed_ratios.shape

(175, 8)

In [4]:
existence = mixed_ratios[:, :] > 0
existence = existence.astype(int)
print(existence.shape)

(175, 8)


In [5]:
df

,7x7x7_Si_001000,7x7x7_Al_001001,7x7x7_H_001002,7x7x7_Na_001003,7x7x7_O_001004,7x7x7_Fe_001005,7x7x7_Mg_001006,7x7x7_C_001007,7x7x7_SiO2_001008,7x7x7_Al2O3_001009,...,7x7x7_C_0600_Feldspar_Fill_003142,7x7x7_C_1000_Feldspar_Fill_003143,7x7x7_C_2000_Feldspar_Fill_003144,7x7x7_C_3000_Feldspar_Fill_003145,7x7x7_C_4000_Feldspar_Fill_003146,7x7x7_C_5000_Feldspar_Fill_003147,7x7x7_C_6000_Feldspar_Fill_003148,7x7x7_C_7000_Feldspar_Fill_003149,7x7x7_C_8000_Feldspar_Fill_003150,7x7x7_C_9000_Feldspar_Fill_003151
0,8.404120e-04,1.064090e-03,1.469400e-05,7.596800e-04,1.584100e-05,1.643980e-03,9.120430e-04,8.543980e-04,8.642660e-04,9.635360e-04,...,8.708300e-04,8.658780e-04,8.508560e-04,8.344220e-04,8.186930e-04,7.965010e-04,7.680210e-04,7.302290e-04,6.736890e-04,5.929290e-04
1,6.850000e-07,8.950000e-07,1.100000e-08,6.040000e-07,1.100000e-08,1.489000e-06,7.630000e-07,6.880000e-07,6.930000e-07,8.150000e-07,...,7.270000e-07,7.320000e-07,6.490000e-07,6.600000e-07,6.210000e-07,6.160000e-07,6.460000e-07,6.000000e-07,5.630000e-07,4.980000e-07
2,6.008020e-04,7.947000e-04,1.100900e-05,5.416380e-04,1.176700e-05,1.330060e-03,6.535110e-04,6.176260e-04,6.090960e-04,6.937200e-04,...,6.173560e-04,6.133160e-04,6.039490e-04,5.949720e-04,5.841830e-04,5.686090e-04,5.504250e-04,5.216640e-04,4.817800e-04,4.259010e-04
3,4.206310e-04,5.775430e-04,7.965000e-06,3.816200e-04,8.406000e-06,1.047290e-03,4.585130e-04,4.379550e-04,4.213330e-04,4.886230e-04,...,4.298310e-04,4.275570e-04,4.219220e-04,4.156770e-04,4.081580e-04,3.991210e-04,3.856030e-04,3.669890e-04,3.391830e-04,2.990280e-04
4,2.027580e-04,2.917970e-04,3.990000e-06,1.885240e-04,4.226000e-06,5.645540e-04,2.268500e-04,2.148290e-04,1.984800e-04,2.372060e-04,...,2.056070e-04,2.042030e-04,2.030920e-04,2.003200e-04,1.971800e-04,1.929110e-04,1.870590e-04,1.780250e-04,1.638170e-04,1.442870e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2.780000e-07,2.610000e-07,3.000000e-09,1.930000e-07,4.000000e-09,1.830000e-07,4.100000e-07,2.230000e-07,2.470000e-07,2.300000e-07,...,2.410000e-07,2.430000e-07,2.320000e-07,2.590000e-07,2.550000e-07,2.560000e-07,2.460000e-07,2.070000e-07,2.080000e-07,1.760000e-07
931,3.110000e-07,2.640000e-07,2.000000e-09,2.010000e-07,3.000000e-09,2.030000e-07,4.150000e-07,2.240000e-07,2.320000e-07,2.300000e-07,...,2.310000e-07,2.200000e-07,2.390000e-07,2.420000e-07,2.170000e-07,2.470000e-07,2.250000e-07,2.130000e-07,2.130000e-07,1.850000e-07
932,2.910000e-07,2.590000e-07,3.000000e-09,2.220000e-07,6.000000e-09,1.770000e-07,4.260000e-07,2.420000e-07,2.380000e-07,2.310000e-07,...,2.600000e-07,2.500000e-07,2.420000e-07,2.420000e-07,2.340000e-07,2.430000e-07,2.150000e-07,2.220000e-07,2.050000e-07,1.790000e-07
933,2.950000e-07,2.370000e-07,4.000000e-09,2.070000e-07,4.000000e-09,1.660000e-07,3.930000e-07,2.140000e-07,2.250000e-07,2.170000e-07,...,2.290000e-07,2.590000e-07,2.240000e-07,2.260000e-07,2.370000e-07,2.450000e-07,2.170000e-07,2.090000e-07,1.990000e-07,1.740000e-07


In [6]:
df.sum()

7x7x7_Si_001000                      0.005878
7x7x7_Al_001001                      0.006964
7x7x7_H_001002                       0.000076
7x7x7_Na_001003                      0.005947
7x7x7_O_001004                       0.000082
                                       ...   
7x7x7_C_5000_Feldspar_Fill_003147    0.004698
7x7x7_C_6000_Feldspar_Fill_003148    0.004416
7x7x7_C_7000_Feldspar_Fill_003149    0.004071
7x7x7_C_8000_Feldspar_Fill_003150    0.003637
7x7x7_C_9000_Feldspar_Fill_003151    0.003084
Length: 175, dtype: float64

In [7]:
fitting_df, carbon_fitting_df, si_fitting_df, c_lines_df, si_lines_df = pfa.PeakFit(
    df,
    bins=bins,
    c_window=[4.2, 4.7], 
    si_window=[1.6, 1.95],
    c_baseline='exp_falloff',
    si_baseline='exp_falloff',
    )

/home/jac2462@uta.edu/Documents/USDASummer2025/Steps/04_ApplyAnalysis/CortesAnalysisPackage/peakfitting/__init__.py:265: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  c_lines_df[df.columns[i]+" baseline"] = c_baseline_lines[i]
/home/jac2462@uta.edu/Documents/USDASummer2025/Steps/04_ApplyAnalysis/CortesAnalysisPackage/peakfitting/__init__.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  c_lines_df[df.columns[i]+" peak"] = c_peak_lines[i]
/home/jac2462@uta.edu/Documents/USDASummer2025/Steps/04_ApplyAnalysis/CortesAnalysisPack

In [8]:
carbon_peak_areas = fitting_df['Carbon Peak Area'].to_numpy()
peak_areas = carbon_peak_areas.reshape(-1, 1)
peak_areas.shape

(175, 1)

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input

input_layer = Input(shape=(935, 1), name='input_data')

# First Conv1D block
x = layers.Conv1D(64, 12, activation='relu', padding='same')(input_layer)
x = layers.BatchNormalization()(x)
# Second Conv1D block
x = layers.Conv1D(32, 12, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
# MaxPooling
x = layers.MaxPooling1D(pool_size=2)(x)
# Flatten
x = layers.Flatten()(x)
# Dense layers with Dropout
x = layers.Dense(935, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.1)(x)

# Shared output for Existence and Mixed ratio
shared = layers.Dense(8, activation=None)(x)

# Existence head
existence_out = layers.Activation('softmax', name='existence')(shared)

# Mixed ratio head
mixed_ratio_out = layers.Activation('softmax', name='mixed_ratio')(shared)

# Peak area head
peak = layers.Dense(128, activation='relu')(x)
peak = layers.Dense(256, activation='relu')(peak)
peak = layers.Dense(128, activation='relu')(peak)
peak_area_out = layers.Dense(1, activation=None, name='peak_area')(peak)

# Model
model = models.Model(inputs=input_layer, outputs=[existence_out, mixed_ratio_out, peak_area_out])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_data          │ (None, 935, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 935, 64)   │        832 │ input_data[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 935, 64)   │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 935, 32)   │     24,608 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 935, 32)   │        128 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 467, 32)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 14944)     │          0 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 935)       │ 13,973,575 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 935)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │    119,808 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     16,512 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     33,024 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8)         │      1,032 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ existence           │ (None, 8)         │          0 │ dense_2[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mixed_ratio         │ (None, 8)         │          0 │ dense_2[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ peak_area (Dense)   │ (None, 1)         │        129 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,202,800 (54.18 MB)

 Trainable params: 14,202,608 (54.18 MB)

 Non-trainable params: 192 (768.00 B)

In [15]:
# kill model in tensorflow
tf.keras.backend.clear_session()

In [10]:
X = df.to_numpy().reshape(-1, 935, 1)

In [11]:
X.shape

(175, 935, 1)

In [12]:
# Use the NumPy arrays for training targets
y_existence = existence  # this is the NumPy array from cell 2
y_mixed_ratio = mixed_ratios  # shape (num_samples, 8)
y_peak_area = peak_areas  # shape (num_samples, 1)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'existence': 'categorical_crossentropy',
        'mixed_ratio': 'categorical_crossentropy',
        'peak_area': 'mse'
    },
    loss_weights={
        'existence': 1.0,
        'mixed_ratio': 1.0,
        'peak_area': 1.0
    },
    metrics={
        'existence': 'accuracy',
        'mixed_ratio': 'accuracy',
        'peak_area': 'mse'
    }
)

# Train the model
history = model.fit(
    X,
    {
        'existence': y_existence,
        'mixed_ratio': y_mixed_ratio,
        'peak_area': y_peak_area
    },
    epochs=100,
    batch_size=16,
    validation_split=0.2
)

Epoch 1/100


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - existence_accuracy: 0.3997 - existence_loss: 9.2846 - loss: 11.2161 - mixed_ratio_accuracy: 0.2213 - mixed_ratio_loss: 1.9535 - peak_area_loss: 0.0023 - peak_area_mse: 0.0022 - val_existence_accuracy: 0.0000e+00 - val_existence_loss: 25.0309 - val_loss: 30.8319 - val_mixed_ratio_accuracy: 0.6571 - val_mixed_ratio_loss: 4.2703 - val_peak_area_loss: 1.0701 - val_peak_area_mse: 1.0701
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - existence_accuracy: 0.3045 - existence_loss: 29.5221 - loss: 34.7488 - mixed_ratio_accuracy: 0.4391 - mixed_ratio_loss: 4.7981 - peak_area_loss: 0.7783 - peak_area_mse: 0.7802 - val_existence_accuracy: 0.0000e+00 - val_existence_loss: 159.3147 - val_loss: 282.9853 - val_mixed_ratio_accuracy: 0.6571 - val_mixed_ratio_loss: 16.5683 - val_peak_area_loss: 97.3882 - val_peak_area_mse: 97.3882
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - existence_accuracy: 0.2878 - existence_loss: 166.6400 - loss: 209.9049 - mixed_r

In [16]:
# test the model
test_X = df.to_numpy().reshape(-1, 935, 1)
test_y_existence = existence  # this is the NumPy array from cell 2
test_y_mixed_ratio = mixed_ratios  # shape (num_samples, 8)
test_y_peak_area = peak_areas  # shape (num_samples, 1)
test_loss, test_existence_loss, test_mixed_ratio_loss, test_peak_area_loss, test_existence_acc, test_mixed_ratio_acc, test_peak_area_mse = model.evaluate(
    test_X,
    {
        'existence': test_y_existence,
        'mixed_ratio': test_y_mixed_ratio,
        'peak_area': test_y_peak_area
    }
)
print(f"Test Loss: {test_loss}")
print(f"Test Existence Loss: {test_existence_loss}")
print(f"Test Mixed Ratio Loss: {test_mixed_ratio_loss}")
print(f"Test Peak Area Loss: {test_peak_area_loss}")
print(f"Test Existence Accuracy: {test_existence_acc}")
print(f"Test Mixed Ratio Accuracy: {test_mixed_ratio_acc}")
print(f"Test Peak Area MSE: {test_peak_area_mse}")

# show the predictions
predictions = model.predict(test_X)
predicted_existence = predictions[0]
predicted_mixed_ratio = predictions[1]
predicted_peak_area = predictions[2]

# Convert predictions to DataFrame for better visualization
predicted_existence_df = pd.DataFrame(predicted_existence, columns=[f'Existence_{i}' for i in range(predicted_existence.shape[1])])
predicted_mixed_ratio_df = pd.DataFrame(predicted_mixed_ratio, columns=[f'Mixed_Ratio_{i}' for i in range(predicted_mixed_ratio.shape[1])])
predicted_peak_area_df = pd.DataFrame(predicted_peak_area, columns=['Predicted_Peak_Area'])
# Combine all predictions into a single DataFrame
# predictions_df = pd.concat([predicted_existence_df, predicted_mixed_ratio_df, predicted_peak_area_df], axis=1)
# # Display the first few rows of the predictions DataFrame
# print(predictions_df.head())

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - existence_accuracy: 0.8463 - existence_loss: 3786537984.0000 - loss: 7608947200.0000 - mixed_ratio_accuracy: 0.0118 - mixed_ratio_loss: 2483133184.0000 - peak_area_loss: 1248543488.0000 - peak_area_mse: 1247021184.0000  


Test Loss: 5341410304.0
Test Existence Loss: 2525927168.0
Test Mixed Ratio Loss: 1197599616.0
Test Peak Area Loss: 1300318848.0
Test Existence Accuracy: 0.9257143139839172
Test Mixed Ratio Accuracy: 0.0057142856530845165
Test Peak Area MSE: 1294990464.0
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [17]:
predicted_mixed_ratio_df

,Mixed_Ratio_0,Mixed_Ratio_1,Mixed_Ratio_2,Mixed_Ratio_3,Mixed_Ratio_4,Mixed_Ratio_5,Mixed_Ratio_6,Mixed_Ratio_7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
170,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
mixed_ratio_df = pd.DataFrame(mixed_ratios, columns=[f'Mixed_Ratio_{i}' for i in range(mixed_ratios.shape[1])])
mixed_ratio_df

,Mixed_Ratio_0,Mixed_Ratio_1,Mixed_Ratio_2,Mixed_Ratio_3,Mixed_Ratio_4,Mixed_Ratio_5,Mixed_Ratio_6,Mixed_Ratio_7
0,1.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
170,0.158928,0.084681,0.0,0.0,0.256390,0.0,0.0,0.5
171,0.127143,0.067745,0.0,0.0,0.205112,0.0,0.0,0.6
172,0.095357,0.050809,0.0,0.0,0.153834,0.0,0.0,0.7
173,0.063571,0.033872,0.0,0.0,0.102556,0.0,0.0,0.8
